# Taller 06: Bases de Datos Vectoriales  
**ICCD753 Recuperación de Información – Prof. Iván Carrera (2024-B, EPN-FIS)**  

**Fecha de Entrega:** Martes 14 de enero de 2025  
**Integrantes:**  
- Dilan Andrade  
- Hernán Sánchez  
- Galo Tarapués  

---

### **Objetivo del Taller**
Implementar y comparar estrategias de recuperación de información utilizando TF-IDF, BM25 y embeddings con bases de datos vectoriales, evaluando su relevancia y eficiencia en un dataset textual real.


# Parte1 Recuperación con TF-IDF 

In [17]:

# Importar librerías necesarias

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [18]:
ruta_dataset = "wiki_movie_plots_deduped.csv"

# Cargamos el CSV en un DataFrame de pandas
df = pd.read_csv(ruta_dataset)

# Eliminamos filas que no tengan un 'Plot' (trama) válido
df = df.dropna(subset=['Plot'])

# Mostramos las primeras filas para verificar la carga
df.head()


,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [19]:
# CPreprocesar e integrar Título y Trama

# Creamos una nueva columna combinando 'Title' y 'Plot'.
# De esta forma, TF-IDF considerará también el título de la película, además de la trama.
df['texto_completo'] = df['Title'].astype(str) + " " + df['Plot'].astype(str)

# Puedes agregar aquí más preprocesamiento si deseas (minúsculas, regex, etc.).
# Por ejemplo:
# df['texto_completo'] = df['texto_completo'].str.lower()
# ... Lematización o stemming, de ser necesario ...


In [20]:

# Creamos el vectorizador TF-IDF con parámetros ajustados:
# - stop_words='english' para remover stopwords en inglés.
# - max_features=20000 para limitar el tamaño del vocabulario.
# - sublinear_tf=True para emplear la escala logarítmica en los conteos de términos.
# - ngram_range=(1, 2) para capturar n-gramas (unigramas y bigramas).
vectorizador_tfidf = TfidfVectorizer(
    stop_words='english',
    max_features=20000,
    sublinear_tf=True,
    ngram_range=(1, 2)
)

# Ajustamos el vectorizador a la nueva columna 'texto_completo'
matriz_tfidf = vectorizador_tfidf.fit_transform(df['texto_completo'])

print("Matriz TF-IDF generada con forma:", matriz_tfidf.shape)


Matriz TF-IDF generada con forma: (34886, 20000)


In [21]:

def buscar_con_tfidf(consulta, numero_resultados=5):
    """
    Dada una consulta en texto, transforma la consulta a su vector TF-IDF
    y calcula la similitud con todos los documentos del dataset.
    Retorna los índices de los documentos más similares y sus puntuaciones.
    """
    # Convertimos la consulta al vector TF-IDF (mismos parámetros que el vectorizador entrenado)
    vector_consulta = vectorizador_tfidf.transform([consulta])

    # Calculamos similitud de coseno con todos los documentos
    similitudes = cosine_similarity(vector_consulta, matriz_tfidf).flatten()

    # Ordenamos los documentos por su puntuación de similitud (descendente)
    indices_ordenados = similitudes.argsort()[::-1][:numero_resultados]
    puntuaciones = similitudes[indices_ordenados]

    return indices_ordenados, puntuaciones


In [22]:

# Ejemplo de consulta
consulta_ejemplo = "dinosaurs"

# Obtenemos los documentos más similares
indices_encontrados, puntuaciones_encontradas = buscar_con_tfidf(consulta_ejemplo, numero_resultados=5)

print(f"Resultados para la consulta: '{consulta_ejemplo}'\n")
for rank, (indice, puntuacion) in enumerate(zip(indices_encontrados, puntuaciones_encontradas), start=1):
    print(f"Rank {rank}")
    print(f"Título (Title): {df.loc[indice, 'Title']}")
    print(f"Similitud: {puntuacion:.4f}")
    # Mostramos los primeros 200 caracteres de la trama (Plot) para no saturar la salida
    print(f"Trama (Plot) (primeros 200 caracteres): {df.loc[indice, 'Plot'][:200]}...")
    print("-"*50)


Resultados para la consulta: 'dinosaurs'

Rank 1
Título (Title): Theodore Rex
Similitud: 0.3514
Trama (Plot) (primeros 200 caracteres): In an alternate futuristic society where humans and anthropomorphic dinosaurs co-exist, a tough police detective named Katie Coltraine (Whoopi Goldberg) is paired with a Tyrannosaurus named Theodore R...
--------------------------------------------------
Rank 2
Título (Title): Dinosaurs! – A Fun-Filled Trip Back in Time!
Similitud: 0.2623
Trama (Plot) (primeros 200 caracteres): The video—with beginning scenes filmed in 1987—begins with a young boy named Phillip (played by Fred Savage) sitting in his bedroom, listening to loud music, and struggling to find an idea for a class...
--------------------------------------------------
Rank 3
Título (Title): We're Back! A Dinosaur's Story
Similitud: 0.2593
Trama (Plot) (primeros 200 caracteres): In present-day New York City, an Eastern bluebird named Buster runs away from his siblings and he meets an intellige

## Reflexión Parte 1: Recuperación con TF-IDF
En esta parte, implementamos un modelo TF-IDF para calcular la relevancia de términos dentro del dataset de tramas de películas. El enfoque resultó efectivo para identificar documentos relacionados directamente con las palabras clave, aunque su capacidad es limitada frente a términos no exactos o con ambigüedades.


# Parte 2 Recuperación con BM25

In [11]:

from elasticsearch import Elasticsearch
import pandas as pd
from tqdm import tqdm
import time

In [12]:
# Conectar con Elasticsearch

# Configuramos la conexión con Elasticsearch
es = Elasticsearch(['http://localhost:9200'])

# Verificamos la conexión
if es.ping():
    print("Conectado a Elasticsearch")
else:
    print("No se pudo conectar a Elasticsearch")

Conectado a Elasticsearch


In [13]:
# Configuración del índice para almacenar los documentos
index_name = "movie_plots"

# Definir el esquema del índice con mapeos para BM25
index_config = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "analysis": {
            "analyzer": {
                "custom_analyzer": {
                    "type": "standard",
                    "stopwords": "_english_"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "Title": {"type": "text", "analyzer": "custom_analyzer"},
            "Plot": {"type": "text", "analyzer": "custom_analyzer"}
        }
    }
}

# Crear el índice en Elasticsearch
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=index_config)
    print(f"Índice '{index_name}' creado exitosamente.")
else:
    print(f"El índice '{index_name}' ya existe.")


Índice 'movie_plots' creado exitosamente.


In [14]:
# Cargar el dataset
ruta_dataset = "wiki_movie_plots_deduped.csv"
df = pd.read_csv(ruta_dataset)

# Filtrar filas con valores nulos en 'Plot'
df = df.dropna(subset=['Plot'])

# Insertar documentos en el índice
for i, row in tqdm(df.iterrows(), total=len(df), desc="Indexando documentos en Elasticsearch"):
    doc = {
        "Title": row['Title'],
        "Plot": row['Plot']
    }
    es.index(index=index_name, id=i, body=doc)

print(f"Se han indexado {len(df)} documentos en el índice '{index_name}'.")


Indexando documentos en Elasticsearch: 100%|██████████| 34886/34886 [35:54<00:00, 16.19it/s]  

Se han indexado 34886 documentos en el índice 'movie_plots'.


In [15]:
def buscar_con_bm25(consulta, numero_resultados=5):
    """
    Realiza una consulta al índice en Elasticsearch utilizando BM25.
    Retorna los documentos más relevantes y sus puntuaciones.
    
    Args:
        consulta (str): Términos de búsqueda.
        numero_resultados (int): Número de resultados a retornar.

    Returns:
        resultados (list): Lista de documentos relevantes con título y trama.
    """
    # Query DSL para realizar la búsqueda
    query = {
        "query": {
            "match": {
                "Plot": consulta  # Buscar coincidencias en la trama
            }
        },
        "size": numero_resultados
    }

    # Realizar la consulta
    respuesta = es.search(index=index_name, body=query)

    # Procesar resultados
    resultados = []
    for hit in respuesta['hits']['hits']:
        resultados.append({
            "Title": hit['_source']['Title'],
            "Plot": hit['_source']['Plot'],
            "Score": hit['_score']
        })
    
    return resultados


In [16]:
# Ejemplo de consulta con BM25
consulta_ejemplo = "dinosaurs"

# Obtener los documentos más relevantes
resultados_bm25 = buscar_con_bm25(consulta_ejemplo, numero_resultados=5)

# Mostrar los resultados
print(f"Resultados para la consulta: '{consulta_ejemplo}'\n")
for rank, resultado in enumerate(resultados_bm25, start=1):
    print(f"Rank {rank}")
    print(f"Título: {resultado['Title']}")
    print(f"Score: {resultado['Score']:.4f}")
    print(f"Trama (primeros 200 caracteres): {resultado['Plot'][:200]}...")
    print("-" * 50)


Resultados para la consulta: 'dinosaurs'

Rank 1
Título: We're Back! A Dinosaur's Story
Score: 12.8403
Trama (primeros 200 caracteres): In present-day New York City, an Eastern bluebird named Buster runs away from his siblings and he meets an intelligent orange Tyrannosaurus named Rex, who is playing golf. He explains to Buster that h...
--------------------------------------------------
Rank 2
Título: Theodore Rex
Score: 11.6915
Trama (primeros 200 caracteres): In an alternate futuristic society where humans and anthropomorphic dinosaurs co-exist, a tough police detective named Katie Coltraine (Whoopi Goldberg) is paired with a Tyrannosaurus named Theodore R...
--------------------------------------------------
Rank 3
Título: Future War
Score: 11.5795
Trama (primeros 200 caracteres): Future War begins aboard a spaceship undergoing a revolt. A man enters and activates an escape pod which travels to Earth and crashes into the Pacific Ocean. The pod contains “The Runaway”, a human sl...


## Reflexión Parte 2: Recuperación con BM25
Usamos Elasticsearch para implementar BM25, lo que permitió una recuperación más robusta que TF-IDF al considerar la saturación de términos y la longitud de los documentos. Esta técnica mostró mejores resultados al priorizar documentos más relevantes para consultas específicas como "dinosaurs".


# Parte 3